In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_validate
import seaborn as sns

from imblearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score


from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
predictors = [
    'L3',
    'L2',
    'L1',
    'CF_CL',
    'CASH_TA',
    'SAL_TA',
    'TL_SAL',
    'INV_COST',
    'INV_SAL',
    'CC_SAL',
    'TA_SAL',
    'TL_TA',
    'CF_TL',
    'CL_TA',
    'NCL_TA',
    'EQ_TL',
    'EQ_TA',
    'CL_CC',
    'ROE',
    'EAT_TA',
    'ROA_BRUTTO',
    'CF_TA',
    'CF_SAL',
    'ROS',
    'EAT_YIE',
    'ROI',
    'ROA_NETTO',
    'LABOR_PRODUCTIVITY',
    'SOCLIAB_SAL']

print(len(predictors))

29


In [3]:
DISTRESS_CRITERIA_1Y_OFF = [
 'DPHZ_vat_registration_cancelled_1y_off',
 'RUZ_indirect_criterion_1y_off',
 'RU_konkurz_1y_off',
 'RU_ostatne_konania_1y_off',
 'RUZ_cancelled_1y_off']


DISTRESS_CRITERIA_2Y_OFF = [
 'DPHZ_vat_registration_cancelled_2y_off',
 'RUZ_indirect_criterion_2y_off',
 'RU_konkurz_2y_off',
 'RU_ostatne_konania_2y_off',
 'RUZ_cancelled_2y_off']



DISTRESS_CRITERIA_3Y_OFF = [
 'DPHZ_vat_registration_cancelled_3y_off',
 'RUZ_indirect_criterion_3y_off',
 'RU_konkurz_3y_off',
 'RU_ostatne_konania_3y_off',
 'RUZ_cancelled_3y_off']

In [4]:
financial_ratios_table_raw = pd.read_csv('../../DATA/FINAL/training_data.csv')
display(financial_ratios_table_raw.shape[0])

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (11,168) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2104439

In [5]:
def combined_issue_3y_off(row):
    if row.DPHZ_vat_registration_cancelled_3y_off == 1: return 1
    if row.RUZ_cancelled_3y_off == 1: return 1
    if row.RU_konkurz_3y_off == 1: return 1
    if row.RU_ostatne_konania_3y_off == 1: return 1
    return 0

def combined_issue_2y_off(row):
    if row.DPHZ_vat_registration_cancelled_2y_off == 1: return 1
    if row.RUZ_cancelled_2y_off == 1: return 1
    if row.RU_konkurz_2y_off == 1: return 1
    if row.RU_ostatne_konania_2y_off == 1: return 1
    return 0


def combined_issue_1y_off(row):
    if row.DPHZ_vat_registration_cancelled_1y_off == 1: return 1
    if row.RUZ_cancelled_1y_off == 1: return 1
    if row.RU_konkurz_1y_off == 1: return 1
    if row.RU_ostatne_konania_1y_off == 1: return 1
    return 0


financial_ratios_table_raw['direct_criterion_3y_off'] = financial_ratios_table_raw.apply(combined_issue_3y_off, axis = 1)
financial_ratios_table_raw['direct_criterion_2y_off'] = financial_ratios_table_raw.apply(combined_issue_2y_off, axis = 1)
financial_ratios_table_raw['direct_criterion_1y_off'] = financial_ratios_table_raw.apply(combined_issue_1y_off, axis = 1)

In [6]:
financial_ratios_table_raw['year'].value_counts()

2014    602568
2021    242626
2020    237437
2019    227048
2018    215784
2017    202819
2016    192149
2015    184008
Name: year, dtype: int64

In [7]:
def get_sample_sizes(data, row_label, column_label):
    samples_df = data.groupby([row_label, column_label])['financial_statement_id'].nunique().to_frame('count').reset_index()
    wide_df = pd.pivot_table(samples_df, index = row_label, columns=column_label, fill_value=0)
    return wide_df

In [8]:
previous_year_data = financial_ratios_table_raw[['entity_ico', 'year'] + predictors]
previous_year_data.columns = ['entity_ico', 'year'] + [col + '_previous' for col in predictors]
previous_year_data['year'] += 1

financial_ratios_table_raw = financial_ratios_table_raw.merge(previous_year_data, on = ['year', 'entity_ico'], how = 'left')

C:\Users\matus.rohal\AppData\Local\Temp\ipykernel_13148\329029966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  previous_year_data['year'] += 1


In [9]:
display(financial_ratios_table_raw.shape[0])
display(financial_ratios_table_raw.query('SAL_TA_previous == SAL_TA_previous').shape[0])

2104449

1336354

In [10]:
records = []


for i, row in financial_ratios_table_raw.iterrows():
    record = [row['financial_statement_id'], row['year']]
    for col in predictors:
        if row[col + '_previous'] == 0:
            value = np.NaN
        else:  
            value = row[col] / row[col + '_previous'] - 1
        record.append(value)
    records.append(record)

yty_growth = pd.DataFrame(records, columns=['financial_statement_id', 'year'] + [col + '_yty' for col in predictors])

yty_growth

,financial_statement_id,year,L3_yty,L2_yty,L1_yty,CF_CL_yty,CASH_TA_yty,SAL_TA_yty,TL_SAL_yty,INV_COST_yty,INV_SAL_yty,CC_SAL_yty,TA_SAL_yty,TL_TA_yty,CF_TL_yty,CL_TA_yty,NCL_TA_yty,EQ_TL_yty,EQ_TA_yty,CL_CC_yty,ROE_yty,EAT_TA_yty,ROA_BRUTTO_yty,CF_TA_yty,CF_SAL_yty,ROS_yty,EAT_YIE_yty,ROI_yty,ROA_NETTO_yty,LABOR_PRODUCTIVITY_yty,SOCLIAB_SAL_yty
0,2382229,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2968194,2015,0.906661,0.910010,-0.178228,1.547166,-0.417187,-0.179660,-0.456889,-0.662391,-0.729434,2.562757,0.219007,-0.554464,3.054642,-0.290785,-0.861640,6.547895,2.362857,-0.757340,0.035344,2.481714,2.479828,0.806488,1.202121,3.244233,3.245598,2.467967,2.481714,-0.307314,-0.298338
2,3194585,2016,-0.635764,-0.636017,-0.592875,-0.920575,0.028997,0.446179,0.577394,0.523838,0.920573,-0.394419,-0.308523,1.281193,-0.912000,1.527470,-0.189434,-0.874984,-0.714815,1.885971,-1.076035,-1.021684,-0.915393,-0.799256,-0.861190,-1.014994,-1.015514,-1.002963,-1.021684,1.147836,1.171925
3,3662172,2017,-0.165770,-0.165535,0.227918,-10.850799,0.267596,0.115679,-0.038116,-0.537083,-0.437369,-0.401188,-0.103685,0.073154,-10.475908,0.032313,0.833610,-0.598646,-0.569285,0.545188,125.149483,53.334428,-14.729143,-11.169110,-10.114727,47.700771,46.968243,393.668134,53.334428,0.613118,0.200741
4,4072051,2018,0.021187,0.020949,-0.166954,-0.941229,-0.367526,-0.061582,-0.019199,0.262113,0.080015,-0.036692,0.065623,-0.079598,-0.951520,-0.240770,1.609951,-0.396737,-0.444756,-0.160131,-0.825840,-0.903299,-0.927080,-0.955379,-0.952451,-0.896953,-0.896466,-0.913207,-0.903299,-0.267346,-0.049297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104444,5127715,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2104445,5270688,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2104446,5062309,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2104447,5052440,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
print(financial_ratios_table_raw.shape[0])
financial_ratios_table_raw = financial_ratios_table_raw.merge(yty_growth, on = ['financial_statement_id', 'year'], how = 'left')
print(financial_ratios_table_raw.shape[0])

2104449


MemoryError: Unable to allocate 25.1 GiB for an array with shape (15, 224957549) and data type int64

In [ ]:
predictors_previous = [col + '_previous' for col in predictors]
predictors_yty = [col + '_yty' for col in predictors]

In [ ]:
issues_columns_list =  ['DPHZ_vat_registration_cancelled',
 'DPHZ_vat_registration_cancelled_1y_off',
 'DPHZ_vat_registration_cancelled_2y_off',
 'DPHZ_vat_registration_cancelled_3y_off',
 'RUZ_indirect_criterion',
 'RUZ_indirect_criterion_1y_off',
 'RUZ_indirect_criterion_2y_off',
 'RUZ_indirect_criterion_3y_off',
 'RU_konkurz',
 'RU_konkurz_1y_off',
 'RU_konkurz_2y_off',
 'RU_konkurz_3y_off',
 'RU_ostatne_konania',
 'RU_ostatne_konania_1y_off',
 'RU_ostatne_konania_2y_off',
 'RU_ostatne_konania_3y_off',
 'RUZ_cancelled',
 'RUZ_cancelled_1y_off',
 'RUZ_cancelled_2y_off',
 'RUZ_cancelled_3y_off']

issues_2y_off_list = [
 'DPHZ_vat_registration_cancelled_2y_off',
 'RUZ_cancelled_2y_off',
 'RUZ_indirect_criterion_2y_off',
 'RU_konkurz_2y_off',
 'RU_ostatne_konania_2y_off',
 'direct_criterion_2y_off'
]



In [ ]:
# 2016: 2014, 2015, 2016
# 2017: 2015, 2016, 2017
# 2018: 2016, 2017, 2018
# 2019: 2017, 2018, 2019

ico_list_3y_history = pd.DataFrame()

for sample_year in 2016, 2017, 2018, 2019:
    master_set = set(financial_ratios_table_raw['entity_ico'].unique())

    for y in sample_year - 2, sample_year - 1, sample_year:
        subset = financial_ratios_table_raw.query('year == @y')
        subset_set = set(subset['entity_ico'].unique())

        master_set = master_set.intersection(subset_set)

    ico_list_3y_history = pd.concat([ico_list_3y_history, financial_ratios_table_raw.query('year == @sample_year and entity_ico in @master_set')[['entity_ico', 'year']]], axis = 0)

    print(f'{sample_year} count', len(master_set))

display(ico_list_3y_history['year'].value_counts())


2016 count 155815
2017 count 161697
2018 count 173544
2019 count 186425


2019    186425
2018    173544
2017    161697
2016    155815
Name: year, dtype: int64

In [ ]:
ico_list_3y_history['has_3y_history'] = 1
financial_ratios_table_raw = financial_ratios_table_raw.merge(ico_list_3y_history, on = ['entity_ico', 'year'], how = 'left')
financial_ratios_table_raw.shape[0]

1679947

In [ ]:
for year in 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021:
    financial_ratios_table_raw.query('year == @year').to_csv(f'../../DATA/FINAL/training_data_{year}.csv', index = False)